# @consumes basics

In [ ]:
# | hide

import asyncio
from IPython.display import Markdown as md

import asyncer

from fastkafka.testing import LocalKafkaBroker, run_script_and_cancel
from fastkafka._testing.local_broker import run_and_match
from fastkafka._components._subprocess import terminate_asyncio_process

You can use `@consumes` decorator to consume messages from Kafka topics. 

In this guide we will create a simple FastKafka app that will consume hello world messages from hello_world topic.

## Import `FastKafka`

To use the `@consumes` decorator, first we need to import the base FastKafka app to create our application.

In [ ]:
# | echo: false

import_fastkafka = """from fastkafka import FastKafka
"""

md(f"```python\n{import_fastkafka}\n```")

```python
from fastkafka import FastKafka

```

## Define the structure of the messages
Next, you need to define the structure of the messages you want to consume from the topic using [pydantic](https://docs.pydantic.dev/). For the guide we'll stick to something basic, but you are free to define any complex message structure you wish in your project, just make sure it can be JSON encoded.

Let's import `BaseModel` and `Field` from pydantic and create a simple `HelloWorld` class containing one string parameter `msg`

In [ ]:
# | echo: false

import_pydantic = """from pydantic import BaseModel, Field
"""
md(f"```python\n{import_pydantic}\n```")

```python
from pydantic import BaseModel, Field

```

In [ ]:
# | echo: false

define_HelloWorld = """
class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )
"""
md(f"```python\n{define_HelloWorld}\n```")

```python

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

```

## Create a base FastKafka app

Now we will create and define a base FastKafka app, replace the \<address_of_your_kafka_bootstrap_server\> with the actual address of your Kafka bootstrap server

In [ ]:
# | echo: false

create_app = """app = FastKafka(bootstrap_servers="<address_of_your_kafka_bootstrap_server>")
"""
md(f"```python\n{create_app}\n```")

```python
app = FastKafka(bootstrap_servers="<address_of_your_kafka_bootstrap_server>")

```

## Create a consumer function and decorate it with `@consumes`

Let's create a consumer function and explain its basic structure:

In [ ]:
# | echo: false

decorate_consumes = """
from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

@app.consumes()
async def on_hello_world(msg: HelloWorld):
    logger.info(f"Got msg: {msg}")
"""
md(f"```python\n{decorate_consumes}\n```")

```python

from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

@app.consumes()
async def on_hello_world(msg: HelloWorld):
    logger.info(f"Got msg: {msg}")

```

The function decorated with the `@consumes` decorator will be called when a message is produced to Kafka.

The message will then be injected into the typed *msg* argument of the function and its type will be used to parse the message.

In this example case, when the message is sent into a *hello_world* topic, it will be parsed into a HelloWorld class and `on_hello_world` function will be called with the parsed class as *msg* argument value.

## Final app

Your app code should look like this:

In [ ]:
# | echo: false

consumes_example = (
    import_fastkafka
    + import_pydantic
    + define_HelloWorld
    + create_app
    + decorate_consumes
)
md(f"```python\n{consumes_example}\n```")

```python
from fastkafka import FastKafka
from pydantic import BaseModel, Field

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )
app = FastKafka(bootstrap_servers="<address_of_your_kafka_bootstrap_server>")

from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

@app.consumes()
async def on_hello_world(msg: HelloWorld):
    logger.info(f"Got msg: {msg}")

```

## Run the app

In [ ]:
script_file = "consumer_example.py"
filename = script_file.split(".py")[0]
cmd = f"fastkafka run --num-workers=1 {filename}:app"
md(f"Now we can run the app. Copy the code above in {script_file} and run it by running\n```shell\n{cmd}\n```")

Now we can run the app. Copy the code above in consumer_example.py and run it by running
```shell
fastkafka run --num-workers=1 consumer_example:app
```

After running the command, you should see this output in your terminal:

In [ ]:
# | hide

with LocalKafkaBroker(topicas=["hello_world"], apply_nest_asyncio=True) as bootstrap_server:
    exit_code, output = await run_script_and_cancel(
            script=consumes_example.replace("<address_of_your_kafka_bootstrap_server>", bootstrap_server),
            script_file=script_file,
            cmd=cmd,
            cancel_after=5,
        )

    assert exit_code == 0, output.decode("UTF-8")

[INFO] fastkafka._testing.local_broker: LocalKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.local_broker: LocalKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.local_broker: LocalKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.local_broker: <class 'fastkafka.testing.LocalKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._test

In [ ]:
# | echo: false

print(output.decode("UTF-8"))

[201768]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[201768]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': '127.0.0.1:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[201768]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[201768]: [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'hello_world'})
[201768]: [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'hello_world'}
[201768]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[201768]: [WARNING] aiokafka.cluster: Topic hello_world is not available during auto-create initialization
[201768]: [INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'hello_world': 0}. 
Starting process cleanup,

## Send the message to kafka topic

Lets check the topic and see if there is a "Hello world!" message in the hello_world topic. In your terminal run:

In [ ]:
# | echo: false

producer_cmd = 'echo {\\"msg\\": \\"Hello world\\"} | kafka-console-producer.sh --topic=hello_world --bootstrap-server=<addr_of_your_kafka_bootstrap_server>'
md(f"```shell\n{producer_cmd}\n```")

```shell
echo {\"msg\": \"Hello world\"} | kafka-console-producer.sh --topic=hello_world --bootstrap-server=<addr_of_your_kafka_bootstrap_server>
```

In [ ]:
# | hide


consumes_example = (
    import_fastkafka
    + import_pydantic
    + define_HelloWorld
    + create_app
    + decorate_consumes
)

with LocalKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    async with asyncer.create_task_group() as task_group:
        consumer_task = task_group.soonify(run_and_match)(
            *cmd.split(" "),
            pattern='Got msg: {"msg": "Hello world!"}',
            timeout=15,
        )

#         consumer_task = task_group.soonify(run_script_and_cancel)(
#             script=consumes_example.replace(
#                 "<address_of_your_kafka_bootstrap_server>", bootstrap_server
#             ),
#             script_file=script_file,
#             cmd=cmd,
#             cancel_after=10,
#         )

        await asyncio.sleep(5)

        producer_task = task_group.soonify(
            asyncio.create_subprocess_shell)(
                cmd=producer_cmd.replace(
                    "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
                ),
                stdout=asyncio.subprocess.PIPE,
                stderr=asyncio.subprocess.PIPE,
            )

#     await terminate_asyncio_process(consumer_task.value)

print(consumer_task.value[1].decode("UTF-8"))

[INFO] fastkafka._testing.local_broker: LocalKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.local_broker: LocalKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.local_broker: LocalKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.local_broker: <class 'fastkafka.testing.LocalKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.local_broker: LocalKafkaBroker.start(): exited.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 210809...
[INFO] fastkafka.

/usr/lib/python3.10/ast.py:50: RuntimeWarning: coroutine 'create_subprocess_shell' was never awaited
  return compile(source, filename, mode, flags,


TimeoutError: 

In [ ]:
await producer_task.value.communicate()

(b'', b'')

You should see the {"msg": "Hello world!"} messages in your topic.

## Choosing a topic

You probably noticed that you didn't define which topic you are sending the message to, this is because the `@produces` decorator determines the topic by default from your function name.
The decorator will take your function name and strip the default "to_" prefix from it and use the rest as the topic name. In this example case, the topic is *hello_world*.

!!! warn \"New topics\"

    Kafka producers and application startup will fail if the topics you are producing to don't yet exist. Before running the app, make sure that the topics are created.

You can choose your custom prefix by defining the `prefix` parameter in produces decorator, like this:

In [ ]:
# | echo: False
decorate_produces_prefix = """
@app.produces(prefix="send_to_")
async def send_to_hello_world(msg: str) -> HelloWorld:
    return HelloWorld(msg=msg)
"""
md(f"```python\n{decorate_produces_prefix}\n```")

In [ ]:
# | hide


produces_example = (
    import_fastkafka
    + import_pydantic
    + define_HelloWorld
    + create_app
    + decorate_produces_prefix
    + define_run.replace("to_hello_world", "send_to_hello_world")
)

with LocalKafkaBroker(topics=["hello_world"], apply_nest_asyncio=True) as bootstrap_server:
    exit_code, output = await run_script_and_cancel(
            script=produces_example.replace("<address_of_your_kafka_bootstrap_server>", bootstrap_server),
            script_file=script_file,
            cmd=cmd,
            cancel_after=5,
        )

    assert exit_code == 0 , output.decode("UTF-8")

    proc = await run_and_match(
        *consumer_cmd.replace(
            "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
        ).split(" "),
        pattern='{"msg": "Hello world!"}',
        timeout=10
    )
    
    await terminate_asyncio_process(proc)

Also, you can define the topic name completely by defining the `topic` in parameter in produces decorator, like this:

In [ ]:
# | echo: False
decorate_produces_topic = """
@app.produces(topic="my_special_topic")
async def to_hello_world(msg: str) -> HelloWorld:
    return HelloWorld(msg=msg)
"""
md(f"```python\n{decorate_produces_topic}\n```")

In [ ]:
# | hide

produces_example = (
    import_fastkafka
    + import_pydantic
    + define_HelloWorld
    + create_app
    + decorate_produces_topic
    + define_run
)

with LocalKafkaBroker(
    topics=["my_special_topic"], apply_nest_asyncio=True
) as bootstrap_server:
    exit_code, output = await run_script_and_cancel(
        script=produces_example.replace(
            "<address_of_your_kafka_bootstrap_server>", bootstrap_server
        ),
        script_file=script_file,
        cmd=cmd,
        cancel_after=5,
    )

    assert exit_code == 0 , output.decode("UTF-8")

    proc = await run_and_match(
        *consumer_cmd.replace("<addr_of_your_kafka_bootstrap_server>", bootstrap_server)
        .replace("hello_world", "my_special_topic")
        .split(" "),
        pattern='{"msg": "Hello world!"}',
        timeout=10
    )

    await terminate_asyncio_process(proc)

## Message data

What you return from your function will be translated to bytes and sent to defined Kafka topic. The typing of the return value is used for generating the documentation for your Kafka app.

In this example case, the return value is HelloWorld class which will be translated into JSON formatted string and then to bytes. The translated data will then be sent to Kafka.